# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 12:46PM,248766,10,Enova-10820,Emerginnova,Released
1,Oct 17 2022 12:46PM,248766,10,Enova-10821,Emerginnova,Released
2,Oct 17 2022 12:25PM,248764,19,8659250,"Emersa Waterbox, LLC",Released
3,Oct 17 2022 12:24PM,248763,19,8659249,"Emersa Waterbox, LLC",Released
4,Oct 17 2022 12:19PM,248762,19,ACG-2102491578,ACG North America LLC,Released
5,Oct 17 2022 12:15PM,248760,10,0086081154,ISDIN Corporation,Released
6,Oct 17 2022 12:15PM,248760,10,0086081155,ISDIN Corporation,Released
7,Oct 17 2022 12:15PM,248760,10,0086081158,ISDIN Corporation,Released
8,Oct 17 2022 12:15PM,248760,10,0086081159,ISDIN Corporation,Released
9,Oct 17 2022 12:15PM,248760,10,0086081160,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,248761,Released,1
27,248762,Released,1
28,248763,Released,1
29,248764,Released,1
30,248766,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248761,NaN,NaN,1.0
248762,NaN,NaN,1.0
248763,NaN,NaN,1.0
248764,NaN,NaN,1.0
248766,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248702,0.0,0.0,6.0
248706,0.0,1.0,11.0
248707,51.0,4.0,0.0
248719,5.0,3.0,6.0
248732,3.0,9.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248702,0,0,6
248706,0,1,11
248707,51,4,0
248719,5,3,6
248732,3,9,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248702,0,0,6
1,248706,0,1,11
2,248707,51,4,0
3,248719,5,3,6
4,248732,3,9,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248702,,,6
1,248706,,1,11
2,248707,51,4,
3,248719,5,3,6
4,248732,3,9,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 12:46PM,248766,10,Emerginnova
2,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC"
3,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC"
4,Oct 17 2022 12:19PM,248762,19,ACG North America LLC
5,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation
29,Oct 17 2022 12:09PM,248757,10,Eywa Pharma Inc.
35,Oct 17 2022 12:05PM,248761,19,"GCH Granules USA, Inc."
36,Oct 17 2022 11:56AM,248758,19,"GUSA Granules USA, Inc."
38,Oct 17 2022 11:52AM,248755,10,Yusen – 3D Matrix
39,Oct 17 2022 11:47AM,248754,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 17 2022 12:46PM,248766,10,Emerginnova,,,2
1,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC",,,1
2,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC",,,1
3,Oct 17 2022 12:19PM,248762,19,ACG North America LLC,,,1
4,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation,,,24
5,Oct 17 2022 12:09PM,248757,10,Eywa Pharma Inc.,,,6
6,Oct 17 2022 12:05PM,248761,19,"GCH Granules USA, Inc.",,,1
7,Oct 17 2022 11:56AM,248758,19,"GUSA Granules USA, Inc.",,,2
8,Oct 17 2022 11:52AM,248755,10,Yusen – 3D Matrix,,,1
9,Oct 17 2022 11:47AM,248754,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 12:46PM,248766,10,Emerginnova,2,,
1,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC",1,,
2,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC",1,,
3,Oct 17 2022 12:19PM,248762,19,ACG North America LLC,1,,
4,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation,24,,
5,Oct 17 2022 12:09PM,248757,10,Eywa Pharma Inc.,6,,
6,Oct 17 2022 12:05PM,248761,19,"GCH Granules USA, Inc.",1,,
7,Oct 17 2022 11:56AM,248758,19,"GUSA Granules USA, Inc.",2,,
8,Oct 17 2022 11:52AM,248755,10,Yusen – 3D Matrix,1,,
9,Oct 17 2022 11:47AM,248754,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 12:46PM,248766,10,Emerginnova,2,,
1,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC",1,,
2,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC",1,,
3,Oct 17 2022 12:19PM,248762,19,ACG North America LLC,1,,
4,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation,24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 12:46PM,248766,10,Emerginnova,2.0,NaN,NaN
1,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
2,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Oct 17 2022 12:19PM,248762,19,ACG North America LLC,1.0,NaN,NaN
4,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation,24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 17 2022 12:46PM,248766,10,Emerginnova,2.0,0.0,0.0
1,Oct 17 2022 12:25PM,248764,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
2,Oct 17 2022 12:24PM,248763,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Oct 17 2022 12:19PM,248762,19,ACG North America LLC,1.0,0.0,0.0
4,Oct 17 2022 12:15PM,248760,10,ISDIN Corporation,24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2736177,61.0,10.0,3.0
12,497450,1.0,4.0,51.0
15,497453,8.0,7.0,8.0
16,497489,2.0,0.0,0.0
19,1492550,15.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2736177,61.0,10.0,3.0
1,12,497450,1.0,4.0,51.0
2,15,497453,8.0,7.0,8.0
3,16,497489,2.0,0.0,0.0
4,19,1492550,15.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,61.0,10.0,3.0
1,12,1.0,4.0,51.0
2,15,8.0,7.0,8.0
3,16,2.0,0.0,0.0
4,19,15.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,61.0
1,12,Released,1.0
2,15,Released,8.0
3,16,Released,2.0
4,19,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19
Status,,,,,
Completed,3.0,51.0,8.0,0.0,0.0
Executing,10.0,4.0,7.0,0.0,0.0
Released,61.0,1.0,8.0,2.0,15.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19
0,Completed,3.0,51.0,8.0,0.0,0.0
1,Executing,10.0,4.0,7.0,0.0,0.0
2,Released,61.0,1.0,8.0,2.0,15.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19
0,Completed,3.0,51.0,8.0,0.0,0.0
1,Executing,10.0,4.0,7.0,0.0,0.0
2,Released,61.0,1.0,8.0,2.0,15.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()